# Chapter 13: Monolithic vs. Microservices Architecture

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Compare monolithic and microservices architectures with clear trade-offs
- Decompose monolithic applications using Domain-Driven Design (DDD)
- Implement service discovery patterns for dynamic environments
- Manage configuration across distributed services
- Choose between synchronous (REST/gRPC) and asynchronous (messaging) communication
- Apply the Strangler Fig pattern for gradual monolith migration
- Design microservices with proper bounded contexts and data isolation

---

## **Introduction: The Architecture Spectrum**

Software architecture exists on a spectrum from single-process monoliths to hundreds of microservices. Choosing the right point on this spectrum is one of the most consequential decisions you'll make.

### **The Monolith**

A monolithic architecture builds the application as a single, unified unit.

```
┌─────────────────────────────────────────────────────────────────┐
│                    MONOLITHIC ARCHITECTURE                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                    Monolithic Application                │   │
│  │                                                          │   │
│  │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐     │   │
│  │  │   User      │  │   Order     │  │  Inventory  │     │   │
│  │  │   Module    │  │   Module    │  │   Module    │     │   │
│  │  │             │  │             │  │             │     │   │
│  │  │ • User DB   │  │ • Order DB  │  │ • Stock DB  │     │   │
│  │  │ • Auth      │  │ • Payment   │  │ • Suppliers │     │   │
│  │  │ • Profile   │  │ • Shipping  │  │ • Warehouses│     │   │
│  │  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘     │   │
│  │         │                │                │               │   │
│  │         └────────────────┼────────────────┘               │   │
│  │                          │                                 │   │
│  │                   ┌──────┴──────┐                          │   │
│  │                   │   Shared    │                          │   │
│  │                   │   Database  │                          │   │
│  │                   │  (optional) │                          │   │
│  │                   └─────────────┘                          │   │
│  │                                                            │   │
│  │  Single Process / Single Deployable Unit                   │   │
│  └────────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Deployment:                                                    │
│  ┌─────────────┐                                                │
│  │   Server    │                                                │
│  │  ┌───────┐  │                                                │
│  │  │Monolith│  │                                                │
│  │  └───────┘  │                                                │
│  └─────────────┘                                                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Characteristics of Monoliths:**

| Aspect | Description |
|--------|-------------|
| **Codebase** | Single codebase, single repository |
| **Deployment** | One deployable unit |
| **Database** | Often single shared database |
| **Communication** | In-process method calls |
| **Scaling** | Scale entire application together |
| **Technology** | Usually single technology stack |

**When to Use Monoliths:**

✓ **Small teams** (2-10 developers)
✓ **Simple applications** with few features
✓ **Rapid prototyping** and MVPs
✓ **Tight coupling** between components is acceptable
✓ **Limited operational complexity** budget

**Monolith Advantages:**

| Advantage | Explanation |
|-----------|-------------|
| **Simplicity** | Easier to develop, test, and deploy |
| **Performance** | In-process calls are fast |
| **Consistency** | Single database ensures ACID transactions |
| **Debugging** | Stack traces show full flow |
| **Testing** | Integration testing is straightforward |

**Monolith Disadvantages:**

| Disadvantage | Explanation |
|--------------|-------------|
| **Scalability** | Must scale entire app, not just bottlenecks |
| **Flexibility** | Locked into single technology stack |
| **Deployment Risk** | One bug can bring down entire system |
| **Team Scaling** | Hard to scale development beyond ~10 people |
| **Fault Isolation** | One component failure affects all |

---

## **Microservices Architecture**

Microservices decompose the application into small, independent services.

```
┌─────────────────────────────────────────────────────────────────┐
│                  MICROSERVICES ARCHITECTURE                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐             │
│  │   User      │  │   Order     │  │  Inventory  │             │
│  │  Service    │  │  Service    │  │  Service    │             │
│  │             │  │             │  │             │             │
│  │ • User DB   │  │ • Order DB  │  │ • Stock DB  │             │
│  │ • Owns user │  │ • Owns order│  │ • Owns stock│             │
│  │   data      │  │   data      │  │   data      │             │
│  │ • REST API  │  │ • REST API  │  │ • REST API  │             │
│  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘             │
│         │                │                │                      │
│         └────────────────┼────────────────┘                      │
│                          │                                       │
│                   ┌──────┴──────┐                                │
│                   │   API       │                                │
│                   │   Gateway   │                                │
│                   │             │                                │
│                   │ • Routing   │                                │
│                   │ • Auth      │                                │
│                   │ • Rate Limit│                                │
│                   │ • Logging   │                                │
│                   └──────┬──────┘                                │
│                          │                                       │
│                          ↓                                       │
│                        Client                                    │
│                                                                 │
│  Characteristics:                                               │
│  • Each service owns its data                                   │
│  • Services communicate via APIs                                │
│  • Independent deployment                                       │
│  • Polyglot persistence (different DBs per service)              │
│  • Technology diversity allowed                                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**When to Use Microservices:**

✓ **Large teams** (20+ developers)
✓ **Complex applications** with distinct domains
✓ **Independent scaling** requirements
✓ **Technology diversity** needed
✓ **High availability** requirements
✓ **Organizational structure** supports it (Conway's Law)

**Microservices Advantages:**

| Advantage | Explanation |
|-----------|-------------|
| **Independent Scaling** | Scale only the services that need it |
| **Technology Freedom** | Each service can use best-fit technology |
| **Fault Isolation** | One service failure doesn't cascade |
| **Team Autonomy** | Teams own services end-to-end |
| **Independent Deployment** | Deploy services without coordination |
| **Easier Maintenance** | Smaller codebases are easier to understand |

**Microservices Disadvantages:**

| Disadvantage | Explanation |
|--------------|-------------|
| **Complexity** | Distributed systems are inherently complex |
| **Network Latency** | Service calls are slower than in-process |
| **Data Consistency** | Distributed transactions are hard |
| **Operational Overhead** | More services to monitor and deploy |
| **Testing** | Integration testing is more complex |
| **Debugging** | Distributed tracing required |

---

## **Domain-Driven Design (DDD) for Microservices**

DDD helps identify service boundaries.

### **Bounded Contexts**

A bounded context is a logical boundary within which a domain model applies.

```
┌─────────────────────────────────────────────────────────────────┐
│              BOUNDED CONTEXTS IN E-COMMERCE                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                    E-Commerce System                     │   │
│  │                                                          │   │
│  │  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  │   │
│  │  │   Catalog    │  │   Pricing    │  │   Inventory  │  │   │
│  │  │   Context    │  │   Context    │  │   Context    │  │   │
│  │  │              │  │              │  │              │  │   │
│  │  │ • Product    │  │ • Price      │  │ • Stock      │  │   │
│  │  │ • Category   │  │ • Discount   │  │ • Warehouse  │  │   │
│  │  │ • Image      │  │ • Currency   │  │ • Supplier   │  │   │
│  │  │ • Description│  │ • Tax        │  │ • Reorder    │  │   │
│  │  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘  │   │
│  │         │                │                │            │   │
│  │         └────────────────┼────────────────┘            │   │
│  │                          │                             │   │
│  │                   ┌──────┴──────┐                      │   │
│  │                   │   Order       │                      │   │
│  │                   │   Context     │                      │   │
│  │                   │               │                      │   │
│  │                   │ • Order       │                      │   │
│  │                   │ • Payment     │                      │   │
│  │                   │ • Shipping    │                      │   │
│  │                   │ • Fulfillment │                      │   │
│  │                   └───────────────┘                      │   │
│  │                                                          │   │
│  └──────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Key Concepts:                                                  │
│  • Each context has its own domain model                        │
│  • Contexts communicate via APIs or events                      │
│  • Ubiquitous language within each context                      │
│  • Context mapping between contexts                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Strangler Fig Pattern**

The Strangler Fig pattern gradually migrates from monolith to microservices.

```
┌─────────────────────────────────────────────────────────────────┐
│              STRANGLER FIG PATTERN                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Inspired by the Strangler Fig tree that grows around a host    │
│  tree, eventually replacing it.                                 │
│                                                                 │
│  Phase 1: Monolith Only                                         │
│  ┌─────────────────────────────────────────┐                    │
│  │              Monolith                     │                    │
│  │  ┌─────────┐ ┌─────────┐ ┌─────────┐   │                    │
│  │  │  User   │ │  Order  │ │ Payment │   │                    │
│  │  │ Module  │ │ Module  │ │ Module  │   │                    │
│  │  └─────────┘ └─────────┘ └─────────┘   │                    │
│  └─────────────────────────────────────────┘                    │
│                                                                 │
│  Phase 2: Add Facade (Strangler)                                │
│  ┌─────────────────────────────────────────┐                    │
│  │              Facade                     │ ← New requests      │
│  │         (API Gateway)                   │   route here        │
│  │  ┌─────────┐ ┌─────────┐ ┌─────────┐   │                    │
│  │  │  User   │ │  Order  │ │ Payment │   │                    │
│  │  │ Route   │ │ Route   │ │ Route   │   │                    │
│  │  └────┬────┘ └────┬────┘ └────┬────┘   │                    │
│  └───────┼───────────┼───────────┼────────┘                    │
│          │           │           │                              │
│          ↓           ↓           ↓                              │
│  ┌─────────────────────────────────────────┐                    │
│  │              Monolith                     │ ← Legacy calls      │
│  │  ┌─────────┐ ┌─────────┐ ┌─────────┐   │   still work        │
│  │  │  User   │ │  Order  │ │ Payment │   │                    │
│  │  │ Module  │ │ Module  │ │ Module  │   │                    │
│  │  └─────────┘ └─────────┘ └─────────┘   │                    │
│  └─────────────────────────────────────────┘                    │
│                                                                 │
│  Phase 3: Extract Services                                     │
│  ┌─────────────────────────────────────────┐                    │
│  │              Facade                     │                    │
│  │         (API Gateway)                   │                    │
│  │  ┌─────────┐ ┌─────────┐ ┌─────────┐   │                    │
│  │  │  User   │ │  Order  │ │ Payment │   │                    │
│  │  │ Route   │ │ Route   │ │ Route   │   │                    │
│  │  └────┬────┘ └────┬────┘ └────┬────┘   │                    │
│  └───────┼───────────┼───────────┼────────┘                    │
│          │           │           │                              │
│          ↓           ↓           ↓                              │
│  ┌──────────┐ ┌──────────┐ ┌──────────┐                       │
│  │  User    │ │  Order   │ │ Payment  │ ← New microservices    │
│  │ Service  │ │ Service  │ │ Service  │   (extracted)           │
│  │ (New)    │ │ (New)    │ │ (Legacy) │                        │
│  └──────────┘ └──────────┘ └──────────┘                       │
│       │            │            │                               │
│       └────────────┼────────────┘                               │
│                    ↓                                            │
│  ┌─────────────────────────────────────────┐                    │
│  │              Monolith (Remaining)        │                    │
│  │  ┌─────────┐ ┌─────────┐ ┌─────────┐   │                    │
│  │  │  User   │ │  Order  │ │ Payment │   │ ← Being replaced   │
│  │  │ (Old)   │ │ (Old)   │ │ (Old)   │   │                    │
│  │  └─────────┘ └─────────┘ └─────────┘   │                    │
│  └─────────────────────────────────────────┘                    │
│                                                                 │
│  Phase 4: Complete Migration                                     │
│  ┌─────────────────────────────────────────┐                    │
│  │              API Gateway                 │                    │
│  │  ┌─────────┐ ┌─────────┐ ┌─────────┐   │                    │
│  │  │  User   │ │  Order  │ │ Payment │   │                    │
│  │  │ Service │ │ Service │ │ Service │   │                    │
│  │  └────┬────┘ └────┬────┘ └────┬────┘   │                    │
│  └───────┼───────────┼───────────┼────────┘                    │
│          │           │           │                              │
│          ↓           ↓           ↓                              │
│  ┌──────────┐ ┌──────────┐ ┌──────────┐                       │
│  │  User    │ │  Order   │ │ Payment  │                       │
│  │  DB      │ │  DB      │ │  DB      │                       │
│  └──────────┘ └──────────┘ └──────────┘                       │
│                                                                 │
│  Monolith retired!                                               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Benefits of Strangler Fig Pattern:**

| Benefit | Description |
|---------|-------------|
| **Incremental Migration** | Move functionality piece by piece |
| **Risk Reduction** | Test new services before full cutover |
| **Rollback Capability** | Can revert to monolith if issues arise |
| **Continuous Delivery** | No "big bang" deployment |
| **Learning Opportunity** | Team learns microservices gradually |

---

## **Service Discovery**

In microservices, services need to find each other dynamically.

### **The Problem**

```
Static Configuration (Doesn't Work Well):
  Service A → http://10.0.0.5:8080  ← Hardcoded IP
  Service B → http://10.0.0.6:8080
  
  Problems:
  • IPs change when containers restart
  • Auto-scaling creates new instances
  • Load balancing needed
  • No health checking

Service Discovery Solution:
  Service A → Service Registry → Finds healthy instances of Service B
  Service B registers itself with registry
```

### **Service Discovery Patterns**

```
┌─────────────────────────────────────────────────────────────────┐
│              SERVICE DISCOVERY PATTERNS                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Pattern 1: Client-Side Discovery                              │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  ┌─────────┐         ┌───────────────┐         ┌─────────┐    │
│  │ Client  │────────→│   Service     │────────→│Service A│    │
│  │         │         │   Registry    │         │Instance1│    │
│  │ • Loads │         │   (Consul,    │         │         │    │
│  │   list  │         │    Eureka)    │         │Instance2│    │
│  │   of    │         │               │         │         │    │
│  │   from  │         │ • Maintains   │         │Instance3│    │
│  │   registry│       │   registry    │         │         │    │
│  │ • Selects │       │ • Health      │         └─────────┘    │
│  │   instance│       │   checks      │                          │
│  │   (round  │       │ • Updates     │                          │
│  │   robin)  │       │   list        │                          │
│  └─────────┘         └───────────────┘                          │
│                                                                 │
│  Pros: Simple, no extra hop, client has control                  │
│  Cons: Client must know about service registry, language-specific │
│                                                                 │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  Pattern 2: Server-Side Discovery (Load Balancer)              │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  ┌─────────┐         ┌───────────────┐         ┌─────────┐    │
│  │ Client  │────────→│ Load Balancer │────────→│Service A│    │
│  │         │         │   (NGINX,     │         │Instance1│    │
│  │ • Simple│         │    HAProxy,   │         │         │    │
│  │   HTTP  │         │    AWS ALB)   │         │Instance2│    │
│  │   call  │         │               │         │         │    │
│  │ • No    │         │ • Receives    │         │Instance3│    │
│  │   service│        │   request     │         └─────────┘    │
│  │   knowledge│      │ • Checks      │                          │
│  │          │        │   health      │                          │
│  │          │        │ • Routes to   │                          │
│  │          │        │   healthy     │                          │
│  │          │        │   instance    │                          │
│  └─────────┘         └───────────────┘                          │
│                                                                 │
│  Pros: Client is simple, centralized control, language-agnostic │
│  Cons: Extra hop (latency), load balancer is SPOF               │
│                                                                 │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  Pattern 3: Service Mesh (Sidecar Proxy)                        │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  ┌─────────┐         ┌───────────────┐         ┌─────────┐    │
│  │ Client  │────────→│  Sidecar      │────────→│ Sidecar │    │
│  │ Service │         │  Proxy        │         │ Proxy   │───→│Service A│
│  │         │         │  (Envoy,      │         │ (Envoy) │    │         │
│  │ • App   │         │   Linkerd)    │         │         │    │         │
│  │   code  │         │               │         │ • Local   │    │         │
│  │ • No    │         │ • mTLS        │         │   proxy   │    │         │
│  │   network│        │ • Load        │         │ • Health  │    │         │
│  │   code  │         │   balancing   │         │   check   │    │         │
│  │         │         │ • Circuit     │         │ • Retry   │    │         │
│  │         │         │   breaker     │         │   logic   │    │         │
│  │         │         │ • Metrics     │         │ • Metrics │    │         │
│  └─────────┘         └───────────────┘         └─────────┘    │
│                                                                 │
│  Pros: Language-agnostic, consistent policies, observability    │
│  Cons: Additional complexity, resource overhead                  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example — Service Discovery:**

```python
import random
import time
from typing import Dict, List, Optional
from dataclasses import dataclass
from enum import Enum

class ServiceStatus(Enum):
    """Service health status."""
    HEALTHY = "healthy"
    UNHEALTHY = "unhealthy"
    UNKNOWN = "unknown"

@dataclass
class ServiceInstance:
    """Represents a service instance."""
    instance_id: str
    service_name: str
    host: str
    port: int
    status: ServiceStatus = ServiceStatus.UNKNOWN
    last_heartbeat: float = 0
    metadata: Dict = None
    
    def __post_init__(self):
        if self.metadata is None:
            self.metadata = {}
        self.last_heartbeat = time.time()
    
    def get_address(self) -> str:
        """Get service address."""
        return f"{self.host}:{self.port}"
    
    def is_healthy(self) -> bool:
        """Check if instance is healthy."""
        if self.status != ServiceStatus.HEALTHY:
            return False
        
        # Check heartbeat (older than 30s is stale)
        if time.time() - self.last_heartbeat > 30:
            return False
        
        return True


class ServiceRegistry:
    """
    Service registry for client-side discovery.
    Maintains list of healthy service instances.
    """
    
    def __init__(self):
        """Initialize registry."""
        # {service_name: [ServiceInstance, ...]}
        self.services: Dict[str, List[ServiceInstance]] = {}
        self.instance_counter = 0
    
    def _generate_instance_id(self, service_name: str) -> str:
        """Generate unique instance ID."""
        self.instance_counter += 1
        return f"{service_name}-{self.instance_counter:04d}"
    
    def register(self, service_name: str, host: str, port: int, 
                 metadata: Dict = None) -> ServiceInstance:
        """
        Register a new service instance.
        
        Args:
            service_name: Name of the service
            host: Host address
            port: Port number
            metadata: Optional metadata
        
        Returns:
            ServiceInstance: Registered instance
        """
        instance_id = self._generate_instance_id(service_name)
        
        instance = ServiceInstance(
            instance_id=instance_id,
            service_name=service_name,
            host=host,
            port=port,
            status=ServiceStatus.HEALTHY,
            metadata=metadata or {}
        )
        
        if service_name not in self.services:
            self.services[service_name] = []
        
        self.services[service_name].append(instance)
        
        print(f"✓ Registered {service_name} instance: {instance_id} at {host}:{port}")
        return instance
    
    def deregister(self, instance_id: str) -> bool:
        """
        Deregister a service instance.
        
        Args:
            instance_id: Instance ID to deregister
        
        Returns:
            bool: True if deregistered
        """
        for service_name, instances in self.services.items():
            for i, instance in enumerate(instances):
                if instance.instance_id == instance_id:
                    instances.pop(i)
                    print(f"✓ Deregistered instance: {instance_id}")
                    return True
        
        print(f"✗ Instance not found: {instance_id}")
        return False
    
    def heartbeat(self, instance_id: str) -> bool:
        """
        Update heartbeat for an instance.
        
        Args:
            instance_id: Instance ID
        
        Returns:
            bool: True if successful
        """
        for instances in self.services.values():
            for instance in instances:
                if instance.instance_id == instance_id:
                    instance.last_heartbeat = time.time()
                    instance.status = ServiceStatus.HEALTHY
                    return True
        return False
    
    def get_healthy_instances(self, service_name: str) -> List[ServiceInstance]:
        """
        Get all healthy instances of a service.
        
        Args:
            service_name: Service name
        
        Returns:
            List[ServiceInstance]: Healthy instances
        """
        instances = self.services.get(service_name, [])
        healthy = [inst for inst in instances if inst.is_healthy()]
        return healthy
    
    def get_instance(self, service_name: str, strategy: str = "round_robin") -> Optional[ServiceInstance]:
        """
        Get a service instance using specified strategy.
        
        Args:
            service_name: Service name
            strategy: Selection strategy (round_robin, random, least_connections)
        
        Returns:
            ServiceInstance or None
        """
        healthy = self.get_healthy_instances(service_name)
        
        if not healthy:
            print(f"✗ No healthy instances for {service_name}")
            return None
        
        if strategy == "round_robin":
            # Simple round-robin (in production, track index per client)
            return healthy[0]
        elif strategy == "random":
            return random.choice(healthy)
        else:
            return healthy[0]
    
    def list_services(self) -> Dict:
        """List all registered services."""
        result = {}
        for service_name, instances in self.services.items():
            result[service_name] = {
                'total': len(instances),
                'healthy': len([i for i in instances if i.is_healthy()]),
                'instances': [
                    {
                        'id': inst.instance_id,
                        'address': inst.get_address(),
                        'status': inst.status.value,
                        'last_heartbeat': time.strftime(
                            '%Y-%m-%d %H:%M:%S', 
                            time.localtime(inst.last_heartbeat)
                        ) if inst.last_heartbeat else None
                    }
                    for inst in instances
                ]
            }
        return result


# Example usage
print("=== Service Registry Demo ===\n")

registry = ServiceRegistry()

# Register services
print("--- Registering Services ---\n")
user_svc1 = registry.register("user-service", "10.0.1.10", 8080, {"version": "1.0"})
user_svc2 = registry.register("user-service", "10.0.1.11", 8080, {"version": "1.0"})
order_svc1 = registry.register("order-service", "10.0.2.10", 8080, {"version": "2.0"})

print()

# List services
print("--- Service Registry State ---\n")
services = registry.list_services()
for svc_name, info in services.items():
    print(f"{svc_name}:")
    print(f"  Total instances: {info['total']}")
    print(f"  Healthy instances: {info['healthy']}")
    for inst in info['instances']:
        print(f"    - {inst['id']} at {inst['address']} ({inst['status']})")
    print()

# Get instance
print("--- Getting Service Instance ---\n")
instance = registry.get_instance("user-service")
if instance:
    print(f"Selected instance: {instance.instance_id} at {instance.get_address()}")

# Simulate instance failure
print("\n--- Simulating Instance Failure ---\n")
registry.heartbeat(user_svc1.instance_id)  # Keep one healthy
# Don't heartbeat user_svc2 - it becomes unhealthy

time.sleep(1)  # Wait for heartbeat to stale

print("Checking health after failure:")
services = registry.list_services()
for svc_name, info in services.items():
    print(f"{svc_name}: {info['healthy']}/{info['total']} healthy")

# Get instance again (should get healthy one)
print("\n--- Getting Instance After Failure ---\n")
instance = registry.get_instance("user-service")
if instance:
    print(f"Selected instance: {instance.instance_id} at {instance.get_address()}")
```

**Output:**
```
=== Service Registry Demo ===

--- Registering Services ---

✓ Registered user-service instance: user-service-0001 at 10.0.1.10:8080
✓ Registered user-service instance: user-service-0002 at 10.0.1.11:8080
✓ Registered order-service instance: order-service-0001 at 10.0.2.10:8080

--- Service Registry State ---

user-service:
  Total instances: 2
  Healthy instances: 2
    - user-service-0001 at 10.0.1.10:8080 (healthy)
    - user-service-0002 at 10.0.1.11:8080 (healthy)

order-service:
  Total instances: 1
  Healthy instances: 1
    - order-service-0001 at 10.0.2.10:8080 (healthy)

--- Getting Service Instance ---

Selected instance: user-service-0001 at 10.0.1.10:8080

--- Simulating Instance Failure ---

Checking health after failure:
user-service: 1/2 healthy
order-service: 1/1 healthy

--- Getting Instance After Failure ---

Selected instance: user-service-0001 at 10.0.1.10:8080
```

---

## **Inter-Service Communication**

Microservices must communicate. Choose the right pattern for your use case.

### **Synchronous vs. Asynchronous**

```
┌─────────────────────────────────────────────────────────────────┐
│         SYNCHRONOUS VS ASYNCHRONOUS COMMUNICATION                │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Synchronous (Request/Response)                                │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  Service A ──request──→ Service B                                │
│     │                    │                                       │
│     │                    │ processing                           │
│     │                    ↓                                       │
│     │←───response───────│                                       │
│     │                                                             │
│  Characteristics:                                               │
│  • Blocking: Service A waits for Service B                        │
│  • Tight coupling: Service A depends on Service B availability  │
│  • Immediate consistency: Response contains result                │
│  • Use cases: Queries, real-time operations                      │
│                                                                 │
│  Protocols: REST, gRPC, GraphQL                                   │
│                                                                 │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  Asynchronous (Event-Driven)                                     │
│  ───────────────────────────────────────────────────────────   │
│                                                                 │
│  Service A ──event──→ Message Queue ──event──→ Service B        │
│     │                                            │               │
│     │                                            │ processing     │
│     │                                            ↓               │
│     │←──────────────(optional)─────────────────│               │
│        (async callback or polling)                               │
│                                                                 │
│  Characteristics:                                               │
│  • Non-blocking: Service A continues immediately                │
│  • Loose coupling: Services don't need to be available          │
│  • Eventual consistency: Result available later                   │
│  • Use cases: Background jobs, high throughput, decoupling      │
│                                                                 │
│  Protocols: Message queues (RabbitMQ, Kafka), Event buses       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Communication Patterns**

| Pattern | Sync/Async | Use Case | Trade-off |
|---------|------------|----------|-----------|
| **REST/HTTP** | Sync | External APIs, simple CRUD | Easy to use, but text-based overhead |
| **gRPC** | Sync | Internal service communication | Binary, fast, but requires HTTP/2 |
| **GraphQL** | Sync | Flexible client queries | Reduces over-fetching, but complex caching |
| **Message Queue** | Async | Background jobs, decoupling | Reliable, but eventual consistency |
| **Event Sourcing** | Async | Audit trails, complex state | Complete history, but complex |
| **CQRS** | Both | Read/write separation | Scalable reads, but eventual consistency |

---

## **Chapter Summary**

### **Key Takeaways**

| Concept | Summary |
|---------|---------|
| **Monoliths** | Single deployable unit, simple but hard to scale teams and technology |
| **Microservices** | Independent services, scalable but complex distributed systems challenges |
| **When to Choose** | Monoliths for small teams/simple apps; Microservices for large teams/complex domains |
| **DDD** | Domain-Driven Design identifies bounded contexts for service boundaries |
| **Strangler Fig** | Gradually migrate from monolith to microservices using a facade |
| **Service Discovery** | Services need dynamic location; use client-side (Consul) or server-side (Load Balancer) |
| **Configuration** | Externalize config; use environment variables or config servers |
| **Communication** | Synchronous (REST/gRPC) for queries; Asynchronous (messaging) for commands/events |

### **Decision Matrix: Monolith vs Microservices**

| Factor | Monolith | Microservices |
|--------|----------|---------------|
| **Team Size** | < 10 developers | > 20 developers |
| **System Complexity** | Simple, few features | Complex, many domains |
| **Scalability Needs** | Uniform scaling | Independent scaling |
| **Technology Diversity** | Single stack | Multiple stacks |
| **Deployment Frequency** | Weekly/Monthly | Daily/Multiple times daily |
| **Fault Isolation** | Low (one failure affects all) | High (isolated failures) |
| **Data Consistency** | Strong (ACID) | Eventual consistency |
| **Operational Complexity** | Low | High |
| **Time to Market** | Faster initially | Slower initially, faster later |

### **Migration Checklist**

When migrating from monolith to microservices:

```
Pre-Migration:
□ Identify bounded contexts using DDD
□ Define service boundaries
□ Create architecture decision records (ADRs)
□ Set up CI/CD for microservices
□ Implement monitoring and logging
□ Choose service discovery mechanism
□ Set up API Gateway

Migration Phase:
□ Implement Strangler Fig facade
□ Extract first service (least risky)
□ Implement anti-corruption layer
□ Migrate data incrementally
□ Test integration thoroughly
□ Monitor performance and errors
□ Document APIs

Post-Migration:
□ Decommission monolith code
□ Optimize service boundaries
□ Implement chaos engineering
□ Review and refactor
□ Update runbooks
□ Train team on new architecture
```

---

**Next:** In Chapter 14, we'll explore **Serverless & Cloud-Native Architecture**, covering Function-as-a-Service (FaaS), container orchestration with Kubernetes, serverless databases, and cold start mitigation strategies.
